In [1]:
%matplotlib qt5
%matplotlib notebook

import numpy as _np
import matplotlib.pyplot as mplt
import scipy.linalg as scylin

import pyaccel
from pymodels import si, bo

mplt.rcParams.update({'grid.linestyle': '--', 'grid.alpha': 0.5, 'axes.grid': True, 'font.size': 14})

In [2]:
acc = si.create_accelerator()
famdata = si.get_family_data(acc)

In [ ]:
# acc = bo.create_accelerator()
# acc.energy = 3e9
# famdata = si.get_family_data(acc)

In [3]:
envelope, cumul_mat, bdiff, fixed_point = pyaccel.optics.calc_beamenvelope(acc, full=True, energy_offset=0.0)

In [4]:
m66 = cumul_mat[-1]
# # To calculate the emittances along the whole ring uncomment the
# # line below:
# m66 = _np.linalg.solve(
#     self._cumul_mat.transpose(0, 2, 1),
#     (self._cumul_mat @ m66).transpose(0, 2, 1)).transpose(0, 2, 1)

# Look at section  D.2 of the Ohmi paper to understand this part of the
# code on how to get the emmitances:

m66 = m66
env = envelope[0]
bdf = bdiff[-1]

# The function numpy.linalg.eig returns the evecs matrix such that:
#    evecs^-1 @ m66 @ evecs = np.diag(evals)
evals, evecs = _np.linalg.eig(m66)
evecsh = evecs.swapaxes(-1, -2).conj()
# Notice that the transformation generated by matrix evecs is the
# inverse of equation 62 of the Ohmi paper.
# So we need to calculate the inverse of evecs:
evecsi = _np.linalg.inv(evecs)
evecsih = evecsi.swapaxes(-1, -2).conj()

# Then, using equation 64, we have:
env0r = evecsi @ env @ evecsih
emits = _np.diagonal(
    env0r, axis1=-1, axis2=-2).real.take([0, 2, 4], axis=-1)

# I don't understand why I have to divide the resulting emittances by
# the norms of evecsi:
norm = _np.linalg.norm(evecsi, axis=-1)
emits /= norm.take([0, 2, 4], axis=-1)

In [5]:
diff = evecsi @ bdf @ evecsih

In [6]:
_np.set_printoptions(precision=7, suppress=False)
-_np.diag(diff)/_np.log(_np.abs(evals))/2 / norm

array([2.4893504e-10-1.3233617e-23j, 2.4893504e-10+1.3233685e-23j,
       2.0614755e-06-4.2073822e-23j, 2.0614755e-06+4.7148007e-23j,
       9.2149696e-38-4.1463335e-54j, 9.2149696e-38+4.1463335e-54j])

In [7]:
emits

array([ 2.4896065e-10,  2.0617528e-06, -4.4303804e-39])

In [28]:
iqs = famdata['QS']['index'][0][0]
acc[iqs].KsL = 0.0046

In [29]:
eqpar = pyaccel.optics.EqParamsFromBeamEnvelope(acc)

In [35]:
spos = _np.array(pyaccel.lattice.find_spos(acc, indices='closed'))
fig = mplt.figure()
gs = mplt.GridSpec(2, 1)
ax = fig.add_subplot(gs[0, 0])
ay = fig.add_subplot(gs[1, 0], sharex=ax)
ax.plot(spos, eqpar.tilt_xyplane/_np.pi * 180, 'o-')
ay.plot(spos, 1e12*(eqpar.envelopes[:, 0, 0]- eqpar.envelopes[:, 2, 2]), color='C1')
ax.set_xlim([0, 518.4/5])

ax.set_ylabel('Angle [°] [m]')
ay.set_xlabel('Position [m]')
ay.set_ylabel(r'$\sigma_x^2 - \sigma_y^2$ [$\mu m^2$]')
mplt.tight_layout()
mplt.show()